In [1]:
import os
import warnings

warnings.simplefilter(action="ignore")
os.environ["GRPC_VERBOSITY"] = "NONE"

# Prerequisites

Please make sure your environmental variables and dependencies are ready to use LLM services.

In [2]:
from dotenv import load_dotenv

load_dotenv("../../.env_api")

True

# Import modules

In [3]:
from langrila.claude import ClaudeFunctionalChat
from langrila.gemini import GeminiFunctionalChat
from langrila.openai import OpenAIFunctionalChat

# Conversation memory

In default, `InMemoryConversationMemory`, `JSONConversationMemory` and `PickleConversationMemory` are available. With extra dependencies, `CosmosConversationMemory` for Azure and `S3ConversationMemory` for AWS are also.

In [4]:
from langrila import InMemoryConversationMemory

## Way to keep conversation history

Things you need to do is just to pass conversation memory module to chat module when initializing or generating. The former way is useful if you want to keep entire history, and the later way is useful to select conversation to keep.

Here is an example of the first way using OpenAI API:

In [5]:
conversation_memory = InMemoryConversationMemory()

chat_openai = OpenAIFunctionalChat(
    api_key_env_name="API_KEY",
    model_name="gpt-4o-2024-08-06",
    conversation_memory=conversation_memory,
)

prompt = "Hello, how are you today?"
response = chat_openai.run(prompt=prompt)

In [6]:
prompt = "Please repeat the last answer."
response = chat_openai.run(prompt=prompt)
print(response.message.content[0].text)

Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?


Here is an example of the second way. In this case, conversation memory doesn't store and insert the history if you don't specify conversation memory in generation method.

In [7]:
chat_openai = OpenAIFunctionalChat(
    api_key_env_name="API_KEY",
    model_name="gpt-4o-2024-08-06",
)

prompt = "Hello, how are you today?"
conversation_memory = InMemoryConversationMemory()
response = chat_openai.run(prompt=prompt, conversation_memory=conversation_memory)

In [8]:
prompt = "Please repeat the last answer."
response = chat_openai.run(prompt=prompt)
print(response.message.content[0].text)

I'm sorry, I can't repeat the last answer as I don't have memory of prior interactions. Can you please provide the question again, and I'll do my best to help you?


But the first turn is stored because conversation memory was passed to generation method.

In [9]:
conversation_memory.load()

[{'role': 'user',
  'content': [{'text': 'Hello, how are you today?'}],
  'name': None},
 {'role': 'assistant',
  'content': [{'text': "Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you with whatever you need. How can I assist you today?"}],
  'name': None}]

# How it works

Conversation memory keeps prompt message and response message with universal message format. If you specify content filter, the masked history is restored in the memory. To keep the history with universal message format allows you to reuse history over the client, for example, save history for OpenAI, then use it for Gemini, even if you calls tools while conversation.

## Multi-turn conversation

It's the demonstration using OpenAI API, but the same for the other client. 

In [10]:
chat_openai = OpenAIFunctionalChat(
    api_key_env_name="API_KEY",
    model_name="gpt-4o-2024-08-06",
)

conversation_memory = InMemoryConversationMemory()

It's an example of multi-turn conversation.

In [11]:
prompt = "Do you know Rude in Final Fantasy VII?"
response = chat_openai.run(
    prompt, model_name="gpt-4o-2024-08-06", conversation_memory=conversation_memory
)
print(response.message.content[0].text)

Yes, Rude is a character from Final Fantasy VII, a popular role-playing game developed by Square Enix. He is a member of the Turks, which is a group of covert operatives working for the Shinra Electric Power Company. Rude is known for his stoic demeanor, loyalty to his partners, particularly his colleague Reno, and his combat prowess. He plays a recurring role throughout the game, often appearing as an adversary to the game's protagonists, Cloud Strife and his party. Despite being an antagonist, Rude is portrayed as having a sense of honor and professionalism.


In [12]:
prompt = "What does he think about Tifa?"
response = chat_openai.run(
    prompt, model_name="gpt-4o-2024-08-06", conversation_memory=conversation_memory
)
print(response.message.content[0].text)

In Final Fantasy VII, Rude is known to have a crush on Tifa Lockhart, one of the main characters and a member of Cloud's party. This information is subtly hinted at in the original game and is more explicitly revealed in supporting materials and adaptations. Despite being on opposing sides, Rude's feelings for Tifa are shown through his reluctance to fight her directly during encounters and his general shyness around her. This aspect of Rude's character adds depth to his personality, highlighting his more human and personable side amid his duties as a Turk.


The contents kept by the conversation memory.

In [13]:
conversation_memory.load()

[{'role': 'user',
  'content': [{'text': 'Do you know Rude in Final Fantasy VII?'}],
  'name': None},
 {'role': 'assistant',
  'content': [{'text': "Yes, Rude is a character from Final Fantasy VII, a popular role-playing game developed by Square Enix. He is a member of the Turks, which is a group of covert operatives working for the Shinra Electric Power Company. Rude is known for his stoic demeanor, loyalty to his partners, particularly his colleague Reno, and his combat prowess. He plays a recurring role throughout the game, often appearing as an adversary to the game's protagonists, Cloud Strife and his party. Despite being an antagonist, Rude is portrayed as having a sense of honor and professionalism."}],
  'name': None},
 {'role': 'user',
  'content': [{'text': 'What does he think about Tifa?'}],
  'name': None},
 {'role': 'assistant',
  'content': [{'text': "In Final Fantasy VII, Rude is known to have a crush on Tifa Lockhart, one of the main characters and a member of Cloud's p

## Multi-turn conversation with tools

It's the demonstration using OpenAI API, but the same for the other client. The tools below is the same as the example in `02.function_calling.ipynb`.

In [14]:
def power_disco_ball(power: bool) -> bool:
    """Powers the spinning disco ball."""
    return f"Disco ball is {'spinning!' if power else 'stopped.'}"


def start_music(energetic: bool, loud: bool, bpm: str) -> str:
    """Play some music matching the specified parameters.

    Args:
      energetic: Whether the music is energetic or not.
      loud: Whether the music is loud or not.
      bpm: The beats per minute of the music.

    Returns: The music property.
    """
    return f"Starting music! {energetic=} {loud=}, {bpm=}"


def dim_lights(brightness: float) -> bool:
    """Dim the lights.

    Args:
      brightness: The brightness of the lights, 0.0 is off, 1.0 is full.
    """
    return f"Lights are now set to {brightness}"

In [15]:
from langrila import ToolConfig, ToolParameter, ToolProperty

tool_configs = [
    ToolConfig(
        name="power_disco_ball",
        description="Powers the spinning disco ball.",
        parameters=ToolParameter(
            properties=[
                ToolProperty(
                    name="power",
                    type="boolean",
                    description="Boolean to spin disco ball.",
                ),
            ],
            required=["power"],
        ),
    ),
    ToolConfig(
        name="start_music",
        description="Play some music matching the specified parameters.",
        parameters=ToolParameter(
            properties=[
                ToolProperty(
                    name="energetic",
                    type="boolean",
                    description="Whether the music is energetic or not.",
                ),
                ToolProperty(
                    name="loud", type="boolean", description="Whether the music is loud or not."
                ),
                ToolProperty(
                    name="bpm",
                    type="string",
                    description="The beats per minute of the music.",
                    enum=["60", "120", "180"],
                ),
            ],
            required=["energetic", "loud", "bpm"],
        ),
    ),
    ToolConfig(
        name="dim_lights",
        description="Dim the lights.",
        parameters=ToolParameter(
            properties=[
                ToolProperty(
                    name="brightness",
                    type="number",
                    description="The brightness of the lights, 0.0 is off, 1.0 is full.",
                ),
            ],
            required=["brightness"],
        ),
    ),
]

Initializing chat module.

In [16]:
chat_openai = OpenAIFunctionalChat(
    api_key_env_name="API_KEY",
    model_name="gpt-4o-mini-2024-07-18",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
    conversation_memory=InMemoryConversationMemory(),
)

First turn.

In [17]:
prompt = "Turn this place into a comfortable party mood!"

response = chat_openai.run(prompt=prompt)

print(response.message.content[0].text)

The party mood is all set! The disco ball is spinning, energetic music is playing at a lively beat of 120 BPM, and the lights are dimmed for that cozy vibe. Get ready to have a great time! 🎉✨


Next turn.

In [18]:
prompt = "Can you make bpm up more?"

response = chat_openai.run(prompt=prompt)

print(response.message.content[0].text)

I've cranked up the music to a pulsating 180 BPM! The energy in the room just skyrocketed. Let’s keep the party going! 🎶🔥


History is added in the prompt as you can find below.

In [19]:
response.prompt

[{'role': 'user',
  'content': [{'type': 'text',
    'text': 'Turn this place into a comfortable party mood!'}],
  'name': 'User'},
 {'content': None,
  'refusal': None,
  'role': 'assistant',
  'audio': None,
  'function_call': None,
  'tool_calls': [{'id': 'call_Xhc4cHGwpijBdUGSxGIEMI5z',
    'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
    'type': 'function'},
   {'id': 'call_GnW8qFpTCg6njvLXwrCknX2N',
    'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}',
     'name': 'start_music'},
    'type': 'function'},
   {'id': 'call_6vtMy5f3FVYMEOjIJrkpURff',
    'function': {'arguments': '{"brightness": 0.5}', 'name': 'dim_lights'},
    'type': 'function'}]},
 {'role': 'tool',
  'tool_call_id': 'call_Xhc4cHGwpijBdUGSxGIEMI5z',
  'name': 'power_disco_ball',
  'content': 'Disco ball is spinning!'},
 {'role': 'tool',
  'tool_call_id': 'call_GnW8qFpTCg6njvLXwrCknX2N',
  'name': 'start_music',
  'content': "Starting music! energetic=True lo

Conversation memory contains tool call and tool use messages.

In [20]:
chat_openai.conversation_memory.load()

[{'role': 'user',
  'content': [{'text': 'Turn this place into a comfortable party mood!'}],
  'name': None},
 {'role': 'function_call',
  'content': [{'name': 'power_disco_ball',
    'args': '{"power": true}',
    'call_id': 'call_Xhc4cHGwpijBdUGSxGIEMI5z'},
   {'name': 'start_music',
    'args': '{"energetic": true, "loud": true, "bpm": "120"}',
    'call_id': 'call_GnW8qFpTCg6njvLXwrCknX2N'},
   {'name': 'dim_lights',
    'args': '{"brightness": 0.5}',
    'call_id': 'call_6vtMy5f3FVYMEOjIJrkpURff'}],
  'name': None},
 {'role': 'function',
  'content': [{'output': 'Disco ball is spinning!',
    'call_id': 'call_Xhc4cHGwpijBdUGSxGIEMI5z',
    'args': '{"power": true}',
    'funcname': 'power_disco_ball'},
   {'output': "Starting music! energetic=True loud=True, bpm='120'",
    'call_id': 'call_GnW8qFpTCg6njvLXwrCknX2N',
    'args': '{"energetic": true, "loud": true, "bpm": "120"}',
    'funcname': 'start_music'},
   {'output': 'Lights are now set to 0.5',
    'call_id': 'call_6vtMy5f

## Multi-turn conversation over the client

Introducing universal message class, we can use OpenAI Chat Completion, Gemini and Claude in a same way and connect them. For universal message system, please see `01.introduction.ipynb`.

In [21]:
# conversation memory allows you to interact with modules for multi-turn conversation.
shared_memory = InMemoryConversationMemory()

gemini = GeminiFunctionalChat(
    api_key_env_name="GEMINI_API_KEY",
    conversation_memory=shared_memory,
)

chat_openai = OpenAIFunctionalChat(
    api_key_env_name="API_KEY",
    conversation_memory=shared_memory,
)

claude = ClaudeFunctionalChat(
    api_key_env_name="ANTHROPIC_API_KEY",
    conversation_memory=shared_memory,
)

Call OpenAI Chat Completion at first.

In [22]:
prompt = "Do you know Rude in Final Fantasy VII?"
response = chat_openai.run(prompt, model_name="gpt-4o-2024-08-06")
print(response.message.content[0].text)

Yes, Rude is a character in the game *Final Fantasy VII*. He is a member of the Turks, a group working for the Shinra Electric Power Company. Known for his quiet demeanor and strong combat abilities, Rude often works alongside his partner, Reno. Characterized by his bald head, sunglasses, and loyalty to his comrades, he plays a recurring role throughout the game as an antagonist to the main characters, though his personality suggests a level of depth and nuanced morality compared to typical villains. Rude also appears in extended media related to *Final Fantasy VII*, such as *Advent Children* and the *Remake*.


After calling OpenAI Chat Completion, conversation history is inherited by Gemini as it is.

In [23]:
prompt = "What does he think about Tifa?"
response = gemini.run(prompt, model_name="gemini-1.5-flash")
print(response.message.content[0].text)

It's hard to say definitively what Rude thinks about Tifa, as his thoughts are never explicitly stated in the games or other official media.  However, we can infer a few things based on his actions and character:

* **Respect for strength:** Rude is a skilled fighter himself, and likely admires Tifa's strength and fighting ability. He might see her as a worthy opponent, even though she is on the opposing side.
* **Professional distance:** As a Turk, Rude is primarily focused on his job and fulfilling Shinra's goals.  He likely views Tifa as an obstacle to those goals, but doesn't seem to hold any personal animosity towards her.
* **Possible recognition of her appeal:** It's also possible that Rude, as a man, might be aware of Tifa's physical attractiveness.  However, this is purely speculation, and he never expresses any romantic interest or attraction to her in the story.

Ultimately, Rude's feelings about Tifa are likely a complex mix of respect for her fighting ability, recognition 

You can see assistant role and message object were automatically transformed for Gemini API in the prompt field.

In [24]:
response.prompt

[parts {
   text: "Do you know Rude in Final Fantasy VII?"
 }
 role: "user",
 parts {
   text: "Yes, Rude is a character in the game *Final Fantasy VII*. He is a member of the Turks, a group working for the Shinra Electric Power Company. Known for his quiet demeanor and strong combat abilities, Rude often works alongside his partner, Reno. Characterized by his bald head, sunglasses, and loyalty to his comrades, he plays a recurring role throughout the game as an antagonist to the main characters, though his personality suggests a level of depth and nuanced morality compared to typical villains. Rude also appears in extended media related to *Final Fantasy VII*, such as *Advent Children* and the *Remake*."
 }
 role: "model",
 parts {
   text: "What does he think about Tifa?"
 }
 role: "user"]

Claude can follow as well.

In [25]:
prompt = "Does Turks members know his feeling?"

response = claude.run(prompt, model_name="claude-3-5-sonnet-20240620")

print(response.message.content[0].text)

Yes, some of the other Turks members are aware of Rude's feelings towards Tifa, particularly his partner Reno. In fact, this aspect of Rude's character is subtly hinted at in the original game and expanded upon in later materials:

1. In the original Final Fantasy VII, during a scene where the Turks confront the party in Gongaga, Reno teases Rude about his crush on Tifa. When listing girls Rude likes, Reno mentions "...That girl from the bar?" to which Rude responds with an embarrassed "......" 

2. This is further reinforced in Before Crisis -Final Fantasy VII- and Crisis Core -Final Fantasy VII-, where it's suggested that Rude has a crush on Tifa but never acts on it due to their opposing sides in the conflict.

3. In Final Fantasy VII Remake, there are a few subtle nods to this, including Rude's reluctance to fight Tifa directly.

So while Rude himself never openly expresses his feelings, it's a known fact among his colleagues, particularly Reno, that he has a soft spot for Tifa. Th

Conversation memory is stored in the universal message format that can be converted to each client message format.

In [26]:
shared_memory.load()

[{'role': 'user',
  'content': [{'text': 'Do you know Rude in Final Fantasy VII?'}],
  'name': None},
 {'role': 'assistant',
  'content': [{'text': 'Yes, Rude is a character in the game *Final Fantasy VII*. He is a member of the Turks, a group working for the Shinra Electric Power Company. Known for his quiet demeanor and strong combat abilities, Rude often works alongside his partner, Reno. Characterized by his bald head, sunglasses, and loyalty to his comrades, he plays a recurring role throughout the game as an antagonist to the main characters, though his personality suggests a level of depth and nuanced morality compared to typical villains. Rude also appears in extended media related to *Final Fantasy VII*, such as *Advent Children* and the *Remake*.'}],
  'name': None},
 {'role': 'user',
  'content': [{'text': 'What does he think about Tifa?'}],
  'name': None},
 {'role': 'assistant',
  'content': [{'text': "It's hard to say definitively what Rude thinks about Tifa, as his thoug

## Multi-turn conversation using tools with multiple client

In [27]:
from langrila import InMemoryConversationMemory

shared_memory = InMemoryConversationMemory()

chat_openai = OpenAIFunctionalChat(
    api_key_env_name="API_KEY",
    model_name="gpt-4o-mini-2024-07-18",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
    conversation_memory=shared_memory,
)

gemini = GeminiFunctionalChat(
    api_key_env_name="GEMINI_API_KEY",
    model_name="gemini-1.5-pro",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
    conversation_memory=shared_memory,
)

claude = ClaudeFunctionalChat(
    model_name="claude-3-5-sonnet-20240620",
    api_key_env_name="ANTHROPIC_API_KEY",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
    conversation_memory=shared_memory,
)

OpenAI at first turn.

In [28]:
prompt = "Turn this place into a comfortable party mood!"

response = chat_openai.run(prompt)

print(response.message.content[0].text)

The party mood is on! The disco ball is spinning, energetic music is playing loud at 120 BPM, and the lights are dimmed to create the perfect atmosphere. Let the fun begin! 🎉💃🕺


Claude can continue conversation without any additional process.

In [29]:
prompt = "Can you make bpm up more?"

response = claude.run(prompt=prompt)

print(response.message.content[0].text)

Great! The music has been updated to play at 180 BPM, which is the highest tempo available. This will create a very lively and energetic atmosphere, perfect for an upbeat party mood. The music is still energetic and loud, matching the exciting vibe we've set up.

Is there anything else you'd like to adjust to enhance the party atmosphere further?


Go back to OpenAI Chat Completion

In [30]:
prompt = "Please turn music more relax."

response = chat_openai.run(prompt)

print(response.message.content[0].text)

The music has been changed to a more relaxed vibe, now playing at 60 BPM. It's softer and calmer to help create a chill atmosphere. If you'd like any more adjustments or additions, just let me know! 🎶✨


Gemini can follow as well

In [31]:
prompt = "Nice. So please go back to the initial bpm."

response = gemini.run(prompt=prompt)

print(response.message.content[0].text)

Alright, the music is now back to 120 BPM, but I've kept it relaxed and at a lower volume as per your previous request. This provides a nice balance between energy and chill.  🎶😊 

Let me know if you have any other requests to fine-tune the party atmosphere! 



If you input non-relavant topic, any tool is not called.

In [32]:
prompt = "What is the weather today?"

response = chat_openai.run(prompt)

print(response.message.content[0].text)

I can't access real-time data, including current weather information. However, you can easily find out the weather by checking a weather website, using a weather app, or asking a voice assistant that can provide live updates. If there's anything else you'd like to know or if you need information on specific weather patterns, feel free to ask!


In [33]:
response.prompt

[{'role': 'user',
  'content': [{'type': 'text',
    'text': 'Turn this place into a comfortable party mood!'}],
  'name': 'User'},
 {'content': None,
  'refusal': None,
  'role': 'assistant',
  'audio': None,
  'function_call': None,
  'tool_calls': [{'id': 'call_MH4dl3cjdJo2ufPm7E7V6KWg',
    'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
    'type': 'function'},
   {'id': 'call_7bL6pPmGeC5Gbwd5KpcVAiah',
    'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}',
     'name': 'start_music'},
    'type': 'function'},
   {'id': 'call_vdPBkgennTS7UilQAlnkrbAZ',
    'function': {'arguments': '{"brightness": 0.5}', 'name': 'dim_lights'},
    'type': 'function'}]},
 {'role': 'tool',
  'tool_call_id': 'call_MH4dl3cjdJo2ufPm7E7V6KWg',
  'name': 'power_disco_ball',
  'content': 'Disco ball is spinning!'},
 {'role': 'tool',
  'tool_call_id': 'call_7bL6pPmGeC5Gbwd5KpcVAiah',
  'name': 'start_music',
  'content': "Starting music! energetic=True lo

# Limitation for the function calling

The output of the tools is required to be serializable when `tool_only` option is False as conversation memory is used for bridging function calling module and chat module internally even if you don't specify conversation memory. So please implement tools to return serializable object if you use tools with no `tool_only` option. If you want to get non-serializable object with tool calling, please specify `tool_only` option. Another way is to use `{Client}FunctionCallingModule`. Here is an example for OpenAI API.

# ConversationMemory modules

## Storing into local json file

In [34]:
from langrila import JSONConversationMemory

conversation_memory = JSONConversationMemory(
    path="conversation_memory.json",
)

chat_openai = OpenAIFunctionalChat(
    api_key_env_name="API_KEY",
    model_name="gpt-4o-2024-08-06",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
    conversation_memory=conversation_memory,
)

In [35]:
prompt = "Turn this place into a comfortable party mood!"

response = chat_openai.run(prompt)

print(response.message.content[0].text)

The disco ball is spinning, energetic and loud music is playing at 120 bpm, and the lights are perfectly dimmed. You're all set for a comfortable and lively party mood! Enjoy! 🎉✨


In [36]:
prompt = "Can you make bpm up more?"

response = chat_openai.run(prompt=prompt)

print(response.message.content[0].text)

I've turned up the beats per minute to 180 for an even more energetic atmosphere! Let's get this party moving! 🎶💃🕺


In [37]:
conversation_memory.load()

[{'role': 'user',
  'content': [{'text': 'Turn this place into a comfortable party mood!'}],
  'name': None},
 {'role': 'function_call',
  'content': [{'name': 'power_disco_ball',
    'args': '{"power": true}',
    'call_id': 'call_sja8z53CVjFi7Y7DXKHp6SBt'},
   {'name': 'start_music',
    'args': '{"energetic": true, "loud": true, "bpm": "120"}',
    'call_id': 'call_rkFrG6ruH5byiGOfyegQiKx6'},
   {'name': 'dim_lights',
    'args': '{"brightness": 0.5}',
    'call_id': 'call_beeZmZKa3l1E9aWydnCH9Ars'}],
  'name': None},
 {'role': 'function',
  'content': [{'output': 'Disco ball is spinning!',
    'call_id': 'call_sja8z53CVjFi7Y7DXKHp6SBt',
    'args': '{"power": true}',
    'funcname': 'power_disco_ball'},
   {'output': "Starting music! energetic=True loud=True, bpm='120'",
    'call_id': 'call_rkFrG6ruH5byiGOfyegQiKx6',
    'args': '{"energetic": true, "loud": true, "bpm": "120"}',
    'funcname': 'start_music'},
   {'output': 'Lights are now set to 0.5',
    'call_id': 'call_beeZmZK

## Pickling conversation history

In [38]:
from langrila import PickleConversationMemory

conversation_memory = PickleConversationMemory(
    path="conversation_memory.pkl",
)

chat_openai = OpenAIFunctionalChat(
    api_key_env_name="API_KEY",
    model_name="gpt-4o-2024-08-06",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
    conversation_memory=conversation_memory,
)

In [39]:
prompt = "Turn this place into a comfortable party mood!"

response = chat_openai.run(prompt)

print(response.message.content[0].text)

The place is now transformed into a comfortable party mood! The disco ball is spinning, the energetic music is playing at 120 BPM, and the lights are dimmed to create the perfect atmosphere. Enjoy the party!


In [40]:
prompt = "Can you make bpm up more?"

response = chat_openai.run(prompt=prompt)

print(response.message.content[0].text)

I've increased the BPM to 180 for an even more energetic vibe. Enjoy the intensified party atmosphere!


In [41]:
conversation_memory.load()

[{'role': 'user',
  'content': [{'text': 'Turn this place into a comfortable party mood!'}],
  'name': None},
 {'role': 'function_call',
  'content': [{'name': 'power_disco_ball',
    'args': '{"power": true}',
    'call_id': 'call_IdJ7Iz2hiNHGqYI7nEoDYDDX'},
   {'name': 'start_music',
    'args': '{"energetic": true, "loud": true, "bpm": "120"}',
    'call_id': 'call_KXHRMuMXYPbPL1mlrxwfIrYL'},
   {'name': 'dim_lights',
    'args': '{"brightness": 0.5}',
    'call_id': 'call_FlKB6xqtyUEe4XR1qlLcLhZk'}],
  'name': None},
 {'role': 'function',
  'content': [{'output': 'Disco ball is spinning!',
    'call_id': 'call_IdJ7Iz2hiNHGqYI7nEoDYDDX',
    'args': '{"power": true}',
    'funcname': 'power_disco_ball'},
   {'output': "Starting music! energetic=True loud=True, bpm='120'",
    'call_id': 'call_KXHRMuMXYPbPL1mlrxwfIrYL',
    'args': '{"energetic": true, "loud": true, "bpm": "120"}',
    'funcname': 'start_music'},
   {'output': 'Lights are now set to 0.5',
    'call_id': 'call_FlKB6xq

## Storing into Azure Cosmos DB

You can easily store conversation memory into the Azure Cosmos DB! Replace paramters below to fit your use case.

Acknowledgement: [@rioriost](https://github.com/rioriost) for base implementation.

In [42]:
from langrila.memory.cosmos import CosmosConversationMemory
from langrila.openai import OpenAIFunctionalChat

conversation_memory = CosmosConversationMemory(
    endpoint_env_name="COSMOS_ENDPOINT",  # env variable names
    key_env_name="COSMOS_KEY",
    db_env_name="COSMOS_DB_NAME",
    container_name="test",
    item_name="sample_conversation",
)


chat_openai = OpenAIFunctionalChat(
    api_key_env_name="API_KEY",
    model_name="gpt-4o-2024-08-06",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
    conversation_memory=conversation_memory,
)

In [43]:
prompt = "Turn this place into a comfortable party mood!"

response = chat_openai.run(prompt)

print(response.message.content[0].text)

The place is now set for a comfortable party mood! The disco ball is spinning, the energetic music at 120 BPM is playing loudly, and the lights are dimmed to just the right level. Enjoy the party!


In [44]:
prompt = "Can you make bpm up more?"

response = chat_openai.run(prompt)

print(response.message.content[0].text)

The music has been cranked up to an energetic 180 BPM! The party vibe is getting even more lively. Enjoy the tunes!


In [45]:
conversation_memory.load()

[{'role': 'user',
  'content': [{'text': 'Turn this place into a comfortable party mood!'}],
  'name': None},
 {'role': 'function_call',
  'content': [{'name': 'power_disco_ball',
    'args': '{"power": true}',
    'call_id': 'call_RAtSdp2Yx5laPK85MBCGHuj9'},
   {'name': 'start_music',
    'args': '{"energetic": true, "loud": true, "bpm": "120"}',
    'call_id': 'call_Ctg2G39no5gvINnW25NiCFVg'},
   {'name': 'dim_lights',
    'args': '{"brightness": 0.5}',
    'call_id': 'call_PtUdFHJAc5DEQXAkxWZi9F5x'}],
  'name': None},
 {'role': 'function',
  'content': [{'output': 'Disco ball is spinning!',
    'call_id': 'call_RAtSdp2Yx5laPK85MBCGHuj9',
    'args': '{"power": true}',
    'funcname': 'power_disco_ball'},
   {'output': "Starting music! energetic=True loud=True, bpm='120'",
    'call_id': 'call_Ctg2G39no5gvINnW25NiCFVg',
    'args': '{"energetic": true, "loud": true, "bpm": "120"}',
    'funcname': 'start_music'},
   {'output': 'Lights are now set to 0.5',
    'call_id': 'call_PtUdFHJ

## Storing into AWS S3

`S3ConversationMemory` is available. Replace Bucket name, object key and credentials with yours to store.

Acknowledgement: [@kun432](https://github.com/kun432)

In [50]:
from langrila.memory.s3 import S3ConversationMemory
from langrila.openai import OpenAIFunctionalChat

# S3ConversationMemory utilizes `boto3.client("s3")` for its operations.
# configuration and credentials are handled in the same manner.
conversation_memory = S3ConversationMemory(
    bucket="langrila-test",
    object_key="conversations/test1.json",  # "PREFIX/OBJECT_KEY" for using prefix
    aws_access_key_id_env_name="AWS_ACCESS_KEY",  # as needed
    aws_secret_access_key_env_name="AWS_SECRET_KEY",  # as needed
)

chat_openai = OpenAIFunctionalChat(
    api_key_env_name="API_KEY",
    model_name="gpt-4o-2024-08-06",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
    conversation_memory=conversation_memory,
)

In [51]:
prompt = "Turn this place into a comfortable party mood!"

response = chat_openai.run(prompt)

print(response.message.content[0].text)

The place is now set for a comfortable party mood! The disco ball is spinning, the lights are dimmed to a cozy level, and energetic music is playing at a perfect volume to keep the vibe lively. Enjoy the party!


In [52]:
prompt = "Can you make bpm up more?"

response = chat_openai.run(prompt)

print(response.message.content[0].text)

The music is now playing at a higher tempo of 180 beats per minute, adding more energy to the party atmosphere. Enjoy the upbeat vibes!


In [53]:
conversation_memory.load()

[{'role': 'user',
  'content': [{'text': 'Turn this place into a comfortable party mood!'}],
  'name': None},
 {'role': 'function_call',
  'content': [{'name': 'power_disco_ball',
    'args': '{"power": true}',
    'call_id': 'call_X6g2qP1ZY1GXNweGwvTucCzx'},
   {'name': 'dim_lights',
    'args': '{"brightness": 0.5}',
    'call_id': 'call_ZsrmFj2AfxkP6qATNIuR4WGY'},
   {'name': 'start_music',
    'args': '{"energetic": true, "loud": false, "bpm": "120"}',
    'call_id': 'call_EPzMYKlm4FRgINgOJXwMDQhr'}],
  'name': None},
 {'role': 'function',
  'content': [{'output': 'Disco ball is spinning!',
    'call_id': 'call_X6g2qP1ZY1GXNweGwvTucCzx',
    'args': '{"power": true}',
    'funcname': 'power_disco_ball'},
   {'output': 'Lights are now set to 0.5',
    'call_id': 'call_ZsrmFj2AfxkP6qATNIuR4WGY',
    'args': '{"brightness": 0.5}',
    'funcname': 'dim_lights'},
   {'output': "Starting music! energetic=True loud=False, bpm='120'",
    'call_id': 'call_EPzMYKlm4FRgINgOJXwMDQhr',
    'ar